In [1]:
#imports
import re

# Task 1 - Table Driven Lexer

In [2]:
class Token:
    def __init__(self, inputValue, inputType):
        self.value = inputValue
        self.type = inputType

class Lexer:
    def __init__(self, input):
        self.code = input
        self.tokenList = []
        self.pos = 0
        self.tokenTable = [
            (r"not|let|return|if|else|for|while|fun|->", "KEYWORDS"),
            (r"[\,\(\)\-\=\:\;\{\}]", "OTHER"),

            (r"\b(float|int|bool|colour)\b", "TYPE"),
            
            (r"\b(true|false)\b", "BOOLEANLITERAL"),
            (r"\d+\.\d+", "FLOATLITERAL"),
            (r"\d+", "INTEGERLITERAL"),
            (r"\#[0-9a-fA-F]{6}", "COLOURLITERAL"),

            (r"\b(__width)\b", "PADWIDTH"),
            (r"\b(__height)\b", "PADHEIGHT"),

            (r"\b(__read)\b", "__READ"),
            (r"\b(__randi)\b", "__RANDI"),

            (r"[\*/]|and|AND", "MULTIPLICATIVEOP"),
            (r"[\+]|or|OR", "ADDITIVEOP"),
            (r"==|!=|<=|>=|[\<\>]", "RELATIONALOP"),

            (r"[a-zA-Z]([a-zA-Z0-9]|_)*", "IDENTIFIER"),

            (r"\b(__print)\b", "__PRINT"),
            (r"\b(__delay)\b", "__DELAY"),
            (r"\b(__pixelr)\b", "__PIXELR"),
            (r"\b(__pixel)\b", "__PIXEL"),
        ]
        self.scanText()
        
        print("L e x e r")
        print("Text: \t" + self.code)
        print("Tokens: " + str(self.tokenList) +"\n")

    
    def scanText(self):
        # loops while less than size of string
        while self.pos < len(self.code):
            # ignores whitespace
            if self.code[self.pos] == " ":
                self.pos += 1
            # if not whitespace
            else:
                value, type = None, None

                # goes through table of patterns
                for pattern, tokenType in self.tokenTable:
                    match = re.match(pattern, self.code[self.pos:])

                    # if they match
                    if match is not None:
                        type = tokenType
                        value = match.group()
                        break
                
                # if value and type remained empyty
                if value == None or type == None:
                    raise Exception("Error in Syntax")
                # if both value and type have been assigned
                else:
                    if type == "KEYWORDS" or type == "OTHER":
                        self.tokenList.append((value.upper(), value))
                        # self.tokenList.append(Token(value, "<"+value+">"))
                    else:
                        self.tokenList.append((type, value))
                        # self.tokenList.append(Token(value, type))
                    self.pos += len(value)

In [3]:
code = "let x:int = __randi 1-10;"
lexer = Lexer(code)

L e x e r
Text: 	let x:int = __randi 1-10;
Tokens: [('LET', 'let'), ('IDENTIFIER', 'x'), (':', ':'), ('TYPE', 'int'), ('=', '='), ('__RANDI', '__randi'), ('INTEGERLITERAL', '1'), ('-', '-'), ('INTEGERLITERAL', '10'), (';', ';')]



# Task 2 - Hand-crafted LL(k) parser

In [39]:
class TerminalNode:
    def __init__(self, name, value):
        self.name = name
        self.value = value


class NTerminalNode:
    def __init__(self, name):
        self.name = name
        self.children = []
        

class Parser:
    def __init__(self, input):
        self.lexer = Lexer(input)
        self.tree = None
        self.parseCode()


    def parseCode(self):
        tuple = self.program()
        
        print("P a r s e r\nAST:")

        tree = self.tree
        self.printAST(tree)
            

    def printAST(self, tree):
        if str(type(tree)) == "<class '__main__.NTerminalNode'>":
            print("     " + str(tree.name))
            for child in tree.children:
                self.printAST(child)
        else:
            print("       " + str(tree.name) + " [" + str(tree.value) + "]")


    def getNextToken(self):
        self.lexer.tokenList.pop(0)


    def padRead(self, tree):
        nTree = NTerminalNode("PADREAD")

        if self.lexer.tokenList[0][0] == "__READ":
            nTree.children.append(TerminalNode("__READ", self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)

            if self.lexer.tokenList[0][0] == ",":
                self.getNextToken()
                self.expr(nTree)
                tree.children.append(nTree)            
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def padRandI(self, tree):
        nTree = NTerminalNode("PADRANDI")

        if self.lexer.tokenList[0][0] == "__RANDI":
            nTree.children.append(TerminalNode("__RANDI", self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)
            tree.children.append(nTree)
        else:
            raise Exception ("Error in Parser")


    def literal(self, tree):
        nTree = NTerminalNode("LITERAL")

        nTree.children.append(TerminalNode(self.lexer.tokenList[0][0], self.lexer.tokenList[0][1])); self.getNextToken()
        tree.children.append(nTree)


    def actualParams(self, tree):
        nTree = NTerminalNode("ACTUALPARAMS")
        self.expr(nTree)

        while self.lexer.tokenList and self.lexer.tokenList[0][0] == ",":
            self.getNextToken()
            self.expr(nTree)
        
        tree.children.append(nTree)

    def functionCall(self, tree):
        nTree = NTerminalNode("FUNCTIONCALL")

        if self.lexer.tokenList[0][0] == "IDENTIFIER":
            nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()
            
            if self.lexer.tokenList[0][0] == "(":
                self.getNextToken()
            
                if self.lexer.tokenList[0][0] != ")":
                    self.actualParams(nTree)
            
                    if self.lexer.tokenList[0][0] == ")":
                        self.getNextToken()
                        tree.children.append(nTree)
                        
                elif self.lexer.tokenList[0][0] != ")":
                    self.getNextToken()
                    tree.children.append(nTree)     

                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def subExpr(self, tree):
        nTree = NTerminalNode("SUBEXPR")

        if self.lexer.tokenList[0][0] == "(":
            self.getNextToken()
            self.expr(nTree)

            if self.lexer.tokenList[0][0] == ")":
                self.getNextToken()
                tree.children.append(nTree)
            
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def unary(self, tree):
        nTree = NTerminalNode("UNARY")

        if self.lexer.tokenList[0][0] in ["-", "NOT"]:
            nTree.children.append(TerminalNode(self.lexer.tokenList[0][0], self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)
            tree.children.append(nTree)

        else:
            raise Exception ("Error in Parser")


    def factor(self, tree):
        nTree = NTerminalNode("FACTOR")

        # LITERAL
        if self.lexer.tokenList[0][0] in ["BOOLEANLITERAL", "INTEGERLITERAL", "FLOATLITERAL", "COLOURLITERAL"]:
            self.literal(nTree)

        # IDENTIFIER & FUNCTIONCALL
        elif self.lexer.tokenList[0][0] == "IDENTIFIER":   
                     
            if len(self.lexer.tokenList) > 1:
                if self.lexer.tokenList[1][0] == "(":
                    self.functionCall(nTree)
                else:
                    nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()
            else:
                nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()

        # SUBEXPR
        elif self.lexer.tokenList[0][0] == "(":
            self.subExpr(nTree)

        # UNARY
        elif self.lexer.tokenList[0][0] == "-" or self.lexer.tokenList[0][0] == "NOT":
            self.unary(nTree)

        # PADRANDI
        elif self.lexer.tokenList[0][0] == "__RANDI":
            self.padRandI(nTree)

        # PADWIDTH
        elif self.lexer.tokenList[0][0] == "PADWIDTH":
            nTree.children.append(TerminalNode("PADWIDTH", self.lexer.tokenList[0][1])); self.getNextToken()

        # PADHEIGHT
        elif self.lexer.tokenList[0][0] == "PADHEIGHT":
            nTree.children.append(TerminalNode("PADHEIGHT", self.lexer.tokenList[0][1])); self.getNextToken()

        #PADREAD
        elif self.lexer.tokenList[0][0] == "__READ":
            self.padRead(nTree)

        else:
            raise Exception ("Error in Parser")
        
        tree.children.append(nTree)
    

    def term(self, tree):
        nTree = NTerminalNode("TERM")
        self.factor(nTree)
        
        while self.lexer.tokenList and self.lexer.tokenList[0][0] == "MULTIPLICATIVEOP":
            nTree.children.append(TerminalNode("MULTIPLICATIVEOP", self.lexer.tokenList[0][1])); self.getNextToken()
            self.factor(nTree)
        
        tree.children.append(nTree)
        

    def simpleExpr(self, tree):
        nTree = NTerminalNode("SIMPLEEXPR")
        self.term(nTree)
        
        while self.lexer.tokenList and self.lexer.tokenList[0][0] == "ADDITIVEOP":
            nTree.children.append(TerminalNode("ADDITIVEOP", self.lexer.tokenList[0][1])); self.getNextToken()
            self.term(nTree)
        
        tree.children.append(nTree)


    def expr(self, tree):
        nTree = NTerminalNode("EXPR")
        self.simpleExpr(nTree)
        
        while self.lexer.tokenList and self.lexer.tokenList[0][0] == "RELATIONALOP":
            nTree.children.append(TerminalNode("RELATIONALOP", self.lexer.tokenList[0][1])); self.getNextToken()
            self.simpleExpr(nTree)
            

        tree.children.append(nTree)


    def assignment(self, tree):
        nTree = NTerminalNode("ASSIGNMENT")

        if self.lexer.tokenList[0][0] == "IDENTIFIER":
            nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()

            if self.lexer.tokenList[0][0] == "=":
                nTree.children.append(TerminalNode("EQUALS", self.lexer.tokenList[0][1])); self.getNextToken()
                self.expr(nTree)
                tree.children.append(nTree)
            
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def variableDecl(self, tree):
        nTree = NTerminalNode("VARIABLEDECL")

        if self.lexer.tokenList[0][0] == "LET":
            nTree.children.append(TerminalNode("LET", self.lexer.tokenList[0][1])); self.getNextToken()
            
            if self.lexer.tokenList[0][0] == "IDENTIFIER":
                nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()
                
                if self.lexer.tokenList[0][0] == ":":
                    self.getNextToken()

                    if self.lexer.tokenList[0][0] == "TYPE":
                        nTree.children.append(TerminalNode("TYPE", self.lexer.tokenList[0][1])); self.getNextToken()

                        if self.lexer.tokenList[0][0] == "=":
                            nTree.children.append(TerminalNode("EQUALS", self.lexer.tokenList[0][1])); self.getNextToken()
                            self.expr(nTree)
                            tree.children.append(nTree)

                        else:
                            raise Exception ("Error in Parser")
                    else:
                        raise Exception ("Error in Parser")
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def printStatement(self, tree):
        nTree = NTerminalNode("PRINTSTATEMENT")

        if self.lexer.tokenList[0][0] == "__PRINT":
            nTree.children.append(TerminalNode("__PRINT", self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)
            tree.children.append(nTree)
        
        else:
            raise Exception ("Error in Parser")


    def delayStatement(self, tree):
        nTree = NTerminalNode("DELAYSTATEMENT")

        if self.lexer.tokenList[0][0] == "__DELAY":
            nTree.children.append(TerminalNode("__DELAY", self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)
            tree.children.append(nTree)

        else:
            raise Exception ("Error in Parser")


    def pixelStatement(self, tree):
        nTree = NTerminalNode("PIXELSTATEMENT")

        if self.lexer.tokenList[0][0] == "__PIXELR":
            nTree.children.append(TerminalNode("__PIXELR", self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)

            if self.lexer.tokenList[0][0] == ",":
                self.getNextToken()
                self.expr(nTree)

                if self.lexer.tokenList[0][0] == ",":
                    self.getNextToken()
                    self.expr(nTree)

                    if self.lexer.tokenList[0][0] == ",":
                        self.getNextToken()
                        self.expr(nTree)

                        if self.lexer.tokenList[0][0] == ",":
                            self.getNextToken()
                            self.expr(nTree)

                            tree.children.append(nTree)
                        
                        else:
                            raise Exception ("Error in Parser")
                    else:
                        raise Exception ("Error in Parser")
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        
        elif self.lexer.tokenList[0][0] == "__PIXEL":
            nTree.children.append(TerminalNode("__PIXEL", self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)

            if self.lexer.tokenList[0][0] == ",":
                self.getNextToken()
                self.expr(nTree)

                if self.lexer.tokenList[0][0] == ",":
                    self.getNextToken()
                    self.expr(nTree)

                    tree.children.append(nTree)
                
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def rtrnStatement(self, tree):
        nTree = NTerminalNode("RTRNSTATEMENT")

        if self.lexer.tokenList[0][0] == "RETURN":
            nTree.children.append(TerminalNode("RETURN", self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)

            tree.children.append(nTree)


    def ifStatement(self, tree):
        nTree = NTerminalNode("IFSTATEMENT")

        if self.lexer.tokenList[0][0] == "IF":
            nTree.children.append(TerminalNode("IF", self.lexer.tokenList[0][1])); self.getNextToken()

            if self.lexer.tokenList[0][0] == "(":
                self.getNextToken()
                self.expr(nTree)
                
                if self.lexer.tokenList[0][0] == ")":
                    self.getNextToken()
                    self.block(nTree)
                    
                    if self.lexer.tokenList and self.lexer.tokenList[0][0] == "ELSE":
                        nTree.children.append(TerminalNode("ELSE", self.lexer.tokenList[0][1])); self.getNextToken()

                        self.block(nTree)

                        tree.children.append(nTree)

                    else:
                        tree.children.append(nTree)
                
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")                


    def forStatement(self, tree):
        nTree = NTerminalNode("FORSTATEMENT")

        if self.lexer.tokenList[0][0] == "FOR":
            nTree.children.append(TerminalNode("FOR", self.lexer.tokenList[0][1])); self.getNextToken()

            if self.lexer.tokenList[0][0] == "(":
                self.getNextToken()

                if self.lexer.tokenList[0][0] == "LET":
                    self.variableDecl(nTree)

                    if self.lexer.tokenList[0][0] == ";":
                        self.getNextToken()
                        self.expr(nTree)
                        self.getNextToken()

                        if self.lexer.tokenList[0][0] == "IDENTIFIER":
                            self.assignment(nTree)
                            
                            if self.lexer.tokenList[0][0] == ")":
                                self.getNextToken()
                                self.block(nTree)
                                tree.children.append(nTree)
                            
                            else:
                                raise Exception ("Error in Parser")
                            
                        elif self.lexer.tokenList[0][0] == ")":
                            self.getNextToken()
                            self.block(nTree)
                            tree.children.append(nTree)
                        
                        else:
                            raise Exception ("Error in Parser")
                    else:
                        raise Exception ("Error in Parser")
                        
                elif self.lexer.tokenList[0][0] == ";":
                    self.getNextToken()
                    self.expr(nTree)
                    self.getNextToken()

                    if self.lexer.tokenList[0][0] == "IDENTIFIER":
                        self.assignment(nTree)
                        
                        if self.lexer.tokenList[0][0] == ")":
                            self.getNextToken()
                            self.block(nTree)
                            tree.children.append(nTree)
                        
                        else:
                            raise Exception ("Error in Parser")
                        
                    elif self.lexer.tokenList[0][0] == ")":
                        self.getNextToken()
                        self.block(nTree)
                        tree.children.append(nTree)
                    
                    else:
                        raise Exception ("Error in Parser")
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def whileStatement(self, tree):
        nTree = NTerminalNode("WHILESTATEMENT")

        if self.lexer.tokenList[0][0] == "WHILE":
            nTree.children.append(TerminalNode("WHILE", self.lexer.tokenList[0][1])); self.getNextToken()

            if self.lexer.tokenList[0][0] == "(":
                self.getNextToken()
                self.expr(nTree)

                if self.lexer.tokenList[0][0] == ")":
                    self.getNextToken()
                    self.block(nTree)
                    tree.children.append(nTree)
                
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def formalParam(self, tree):
        nTree = NTerminalNode("FORMALPARAM")

        if self.lexer.tokenList[0][0] == "IDENTIFIER":
            nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()

            if self.lexer.tokenList[0][0] == ":":
                self.getNextToken()

                if self.lexer.tokenList[0][0] == "TYPE":
                    nTree.children.append(TerminalNode("TYPE", self.lexer.tokenList[0][1])); self.getNextToken()
                    
                    tree.children.append(nTree)
                
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")

   
    def formalParams(self, tree):
        nTree = NTerminalNode("FORMALPARAMS")

        self.formalParam(nTree)

        while self.lexer.tokenList and self.lexer.tokenList[0][0] == ",":
            self.getNextToken()
            self.formalParam(nTree)

        tree.children.append(nTree)

   
    def functionDecl(self,tree):
        nTree = NTerminalNode("FUNCTIONDECL")

        if self.lexer.tokenList[0][0] == "FUN":
            nTree.children.append(TerminalNode("FUN", self.lexer.tokenList[0][1])); self.getNextToken()
            
            if self.lexer.tokenList[0][0] == "IDENTIFIER":
                nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()

                if self.lexer.tokenList[0][0] == "(":
                    self.getNextToken()

                    if self.lexer.tokenList[0][0] == "IDENTIFIER":
                        self.formalParams(nTree)

                        if self.lexer.tokenList[0][0] == ")":
                            self.getNextToken()
                             
                            if self.lexer.tokenList[0][0] == "->":
                                nTree.children.append(TerminalNode("ARROW", self.lexer.tokenList[0][1])); self.getNextToken()

                                if self.lexer.tokenList[0][0] == "TYPE":
                                    nTree.children.append(TerminalNode("TYPE", self.lexer.tokenList[0][1])); self.getNextToken()
                                    self.block(nTree)
                                    tree.children.append(nTree)
                                
                                else:
                                    raise Exception ("Error in Parser")
                            else:
                                raise Exception ("Error in Parser")
                        else:
                            raise Exception ("Error in Parser")
                                
                    elif self.lexer.tokenList[0][0] == ")":
                        self.getNextToken()
                            
                        if self.lexer.tokenList[0][0] == "->":
                            nTree.children.append(TerminalNode("ARROW", self.lexer.tokenList[0][1])); self.getNextToken()

                            if self.lexer.tokenList[0][0] == "TYPE":
                                nTree.children.append(TerminalNode("TYPE", self.lexer.tokenList[0][1])); self.getNextToken()
                                self.block(nTree)

                                tree.children.append(nTree)
                            
                            else:
                                raise Exception ("Error in Parser")
                        else:
                            raise Exception ("Error in Parser")
                    else:
                        raise Exception ("Error in Parser")
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")
                            
                            
    def statement(self, tree): 
        nTree = NTerminalNode("STATEMENT")

        # VARIABLEDECL
        if self.lexer.tokenList[0][0] == "LET":
            self.variableDecl(nTree)

            if self.lexer.tokenList[0][0] == ";":
                self.getNextToken()
            else:
                raise Exception ("Error in Parser")

        # ASSIGNMENT
        elif self.lexer.tokenList[0][0] == "IDENTIFIER":
            self.assignment(nTree)
            
            if self.lexer.tokenList and self.lexer.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error in Parser")

        # PRINTSTATEMENT
        elif self.lexer.tokenList[0][0] == "__PRINT":
            self.printStatement(nTree)

            if self.lexer.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error in Parser")

        # DELAYSTATEMENT
        elif self.lexer.tokenList[0][0] == "__DELAY":
            self.delayStatement(nTree)

            if self.lexer.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error in Parser")

        # PIXELSTATEMENT
        elif self.lexer.tokenList[0][0] in ["__PIXEL", "__PIXELR"]:
            self.pixelStatement(nTree)

            if self.lexer.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error in Parser")

        # IFSTATEMENT
        elif self.lexer.tokenList[0][0] == "IF":
            self.ifStatement(nTree)

        # FORSTATEMENT
        elif self.lexer.tokenList[0][0] == "FOR":
            self.forStatement(nTree)

        # WHILESTATEMENT
        elif self.lexer.tokenList[0][0] == "WHILE":
            self.whileStatement(nTree)

        elif self.lexer.tokenList[0][0] == "RETURN":
            self.rtrnStatement(nTree)
            
            if self.lexer.tokenList[0][0] == ";":
                self.getNextToken()

        # FUNCTIONDECL
        elif self.lexer.tokenList[0][0] == "FUN":
            self.functionDecl(nTree)

        # BLOCK
        elif self.lexer.tokenList[0][0] == "{":
            self.block(nTree)
            
        else:
            raise Exception ("Error in STATEMENT")
        
        tree.children.append(nTree)

   
    def block(self, tree):
        nTree = NTerminalNode("BLOCK")

        if self.lexer.tokenList[0][0] == "{":
            self.getNextToken()

            if self.lexer.tokenList[0][0] != "}":
                
                while self.lexer.tokenList and self.lexer.tokenList[0][0] != "}":
                    (self.statement(nTree))

                self.getNextToken()
            elif self.lexer.tokenList[0][0] == "}":
                self.getNextToken()
            else:
                raise Exception ("Error in Parser")
            
            tree.children.append(nTree)
        else:
            raise Exception ("Error in Parser")

   
    def program(self):
        tree = NTerminalNode("PROGRAM")
        
        while self.lexer.tokenList:
            (self.statement(tree))

        self.tree = tree

In [40]:
code = "fun XGreaterY(x:int, u:int) -> bool { let ans:bool = true; if(y>x) {ans = false;} }"
code += "fun XGreaterY_2(x:int, y:int) -> bool{ return x>y; }"
code += "funAverageOfTwo(x:int, y:int) -> float {let t0:int = x + y; let t1:int = t0 / 2; return t1;} "
code += "{let x:int = 5 + 10;}"
code = "let x:int = 5.1;"
parser = Parser(code)

L e x e r
Text: 	let x:int = 5.1;
Tokens: [('LET', 'let'), ('IDENTIFIER', 'x'), (':', ':'), ('TYPE', 'int'), ('=', '='), ('FLOATLITERAL', '5.1'), (';', ';')]

P a r s e r
AST:
     PROGRAM
     STATEMENT
     VARIABLEDECL
       LET [let]
       IDENTIFIER [x]
       TYPE [int]
       EQUALS [=]
     EXPR
     SIMPLEEXPR
     TERM
     FACTOR
     LITERAL
       FLOATLITERAL [5.1]


# Task 3 - AST XML Generation Pass

# Task 4 - Semantic Analysis Pass


# Task 5 - PixIR Code Generation Pass
